<a href="https://colab.research.google.com/github/Stevark/PBICheck_3/blob/master/PBIcheck_Kravets_Dmitriy_DZ3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
link = 'https://haykinanc.github.io/mts_users.io/'
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

result = requests.get(link)
soup = BeautifulSoup(result.content, 'html.parser')
result = []
#print (soup)

root_id = soup.find('div',{"id": "root"}) # поиск ID Root, где все карточки класса user
#print (root_id)
for element in root_id.select('.user'): # перебор всех сторок класса user
      First_name = element.select('p')[1].text#разбивка по тегу абзаца, через один, пропуская наименования
      Last_name = element.select('p')[3].text
      Late_pay = element.select('p')[5].text
      Time_min_prev_month = element.select('p')[7].text
      Time_min_month = element.select('p')[9].text
      Options = element.select('p')[11].text
      Traffic = element.select('p')[13].text
      
      lp = 0
      if Late_pay == "ДА":
          lp = 1
      else:
          lp = 0

      z = (0.01 - (0.7 * lp) - (0.00015 * int(Time_min_prev_month)) - (0.00012 * int(Time_min_month)) - (0.00013 * int(Options)) - (0.00022 * int(Traffic)))
      print(z) # сверял кооэфициент, т.к. в BI получились слегка другие данные, судя по всему из-за вычисления экспоненты. Коффциенты идентичны.
      chance_outflow = math.exp(z) / (1 + math.exp(z))
      result.append({ 
        'Имя': First_name,
        'Фамилия': Last_name,
        'Наличие просрочки по платежам': Late_pay,
        'Продолжительность разговоров в прошлом месяце в минутах': Time_min_prev_month,
        'Продолжительность разговоров в этом месяце в минутах': Time_min_month,
        'Количество подключенных новых опций в штуках': Options,
        'Исходящий трафик в мегабайтах за этот месяц': Traffic,
        'Вероятность оттока': chance_outflow
      })
#print (*result,sep='\n')
df_data = pd.DataFrame(result)
df_data.head(30)
df_data.to_csv('out1.csv', encoding="cp1251") #кодировка, иначе кириллица криво отображается в CSV

-1.90132
-1.01991
-1.76743
-1.03328
-1.50917
-2.40528
-1.12619
